In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import transformers
import pickle
import requests
import pandas as pd
import requests
from datetime import date
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import pipeline

In [4]:
def bitcoin_price():
    url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd"
    response = requests.get(url)
    data = response.json()
    bitcoin_price = data["bitcoin"]["usd"]
    return bitcoin_price

In [5]:
def sentiment(title):
  model_path = '/content/drive/MyDrive/Aylin Irem Kara - FYP/Colab/btc_bert_model'
  tokenizer_path = '/content/drive/MyDrive/Aylin Irem Kara - FYP/Colab/btc_bert_tokenizer'
  model = AutoModelForSequenceClassification.from_pretrained(model_path)
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
  tokens = tokenizer(title, truncation=True, padding=True, return_tensors='pt')
  outputs = model(**tokens)
  logits = outputs.logits
  probabilities = logits.softmax(dim=1)
  predicted_class = probabilities.argmax(dim=1).item()
  if(predicted_class == 2):
      predicted_class = 1
  elif(predicted_class == 1):
      predicted_class = 0
  elif(predicted_class == 0):
      predicted_class = -1
  return predicted_class

In [6]:
#first case:
#path = "/content/drive/MyDrive/Aylin Irem Kara - FYP/All Files For Grad Project/logreg_secondpred_balanced.sav"
path = "/content/drive/MyDrive/Aylin Irem Kara - FYP/All Files For Grad Project/svm_secondpred_balanced.sav"

#second case:
#path = "/content/drive/MyDrive/Aylin Irem Kara - FYP/All Files For Grad Project/rf_secondpred_original.sav"
#path = "/content/drive/MyDrive/Aylin Irem Kara - FYP/All Files For Grad Project/logreg_secondpred_balanced.sav"

In [7]:
def func(title,USDI,dowjones):
  predicted_class = sentiment(title)
  dowjones = Dow_Jones.replace(',', '')
  dowjones = float(dowjones.split('.')[0])
  price = bitcoin_price()
  loaded_model = pickle.load(open(path, 'rb'))
  input = pd.DataFrame({'Score': [predicted_class], 'Price': [price], 'USDI':[USDI], 'Dow_Jones':[dowjones]})
  result = loaded_model.predict(input)

  print("---TODAY BTC PRICE---")
  print(price,"USD")
  print("---TITLE---")
  print(title)
  print("---SENTIMENT---")
  if(predicted_class == 1):
      print("Positive")
  elif(predicted_class == 0):
      print("Neutral")
  elif(predicted_class == -1):
      print("Negative")
  print("---PREDICTION---")
  if(result == -1):
    result = "Tomorrow, BTC price will decrease."
  elif(result == 1):
    result = "Tomorrow, BTC price will increase."
  elif(result == 0):
    result = "Tomorrow, BTC price will be the same as today."
  print(result)

In [10]:
title = "Bitcoin Hovers Around $26K Following Softer-Than-Expected CPI Data" #@param {type:"string"}
USDI = "105.02 " #@param {type:"string"}
Dow_Jones = "33,956.03" #@param {type:"string"}

func(title,USDI,Dow_Jones)

---TODAY BTC PRICE---
25880 USD
---TITLE---
U.S. Inflation Cools in May; Bitcoin Rises
---SENTIMENT---
Neutral
---PREDICTION---
Tomorrow, BTC price will increase.
